# Début d'implémentation du modèle

### Étape 0 : Importations

In [1]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import math
import pandas as pd

### Étape 1 : Récupération des données

In [2]:
corpus = [
    'il est un roi',
    'elle est une reine',
    'il est un homme',
    'elle est une femme',
    'londres est la capitale du royaume uni',
    "berlin est la capitale de l allemagne",
    'paris est la capitale de la france',
]

### Étape 2 : Créer le vocabulaire à partir du corpus de phrases
A faire :
- Que faire des @ ?
- Traiter la ponctuation
- Accents/minuscule 

In [3]:
def tokenize(corpus):
    tokens = [phrase.split() for phrase in corpus]
    return tokens

t_corpus = tokenize(corpus)
t_corpus

[['il', 'est', 'un', 'roi'],
 ['elle', 'est', 'une', 'reine'],
 ['il', 'est', 'un', 'homme'],
 ['elle', 'est', 'une', 'femme'],
 ['londres', 'est', 'la', 'capitale', 'du', 'royaume', 'uni'],
 ['berlin', 'est', 'la', 'capitale', 'de', 'l', 'allemagne'],
 ['paris', 'est', 'la', 'capitale', 'de', 'la', 'france']]

In [5]:
voc = []
for phrase in t_corpus:
    for mot in phrase:
        if mot not in voc:
            voc.append(mot)
voc_size = len(voc)
voc

['il',
 'est',
 'un',
 'roi',
 'elle',
 'une',
 'reine',
 'homme',
 'femme',
 'londres',
 'la',
 'capitale',
 'du',
 'royaume',
 'uni',
 'berlin',
 'de',
 'l',
 'allemagne',
 'paris',
 'france']

### Étape 3 : Créations pairs mots centraux / contexte

In [6]:
mot_index = {w: index for (index, w) in enumerate(voc)}
index_mot = {index: w for (index, w) in enumerate(voc)}

taille_fenetre = 3
index_pairs = []
# On traite chaque phrase.
for phrase in t_corpus:
    indices = [mot_index[mot] for mot in phrase]
    # On traite chaque mot comme un mot central
    for center_word in range(len(indices)):
        # Pour chaque fenetre possible
        for w in range(-taille_fenetre, taille_fenetre + 1):
            context_word = center_word + w
            # On fait attention à ne pas sauter de phrases
            if context_word < 0 or context_word >= len(indices) or center_word == context_word:
                continue
            context_word_ind = indices[context_word]
            index_pairs.append((indices[center_word], context_word_ind))


In [35]:
index_pairs_np = np.array(index_pairs)
index_pairs_np

array([[  0,   1],
       [  0,   2],
       [  0,   3],
       ...,
       [415, 370],
       [415, 629],
       [415,   5]])

### Étape 4 : Création du modèle

In [8]:
#Couche d'entrée
def get_input_layer(word_idx):
    x = torch.zeros(voc_size).float()
    x[word_idx] = 1.0
    return x

# Choix de dimension
embedding_dims = 50
# Initialisation
# Variable : comme Tensor mais avec les valeurs qui changent pendant le traitement
W1 = Variable(torch.randn(embedding_dims, voc_size).float(), requires_grad=True)
W2 = Variable(torch.randn(voc_size, embedding_dims).float(), requires_grad=True)
num_epochs = 10 # "steps" mais sur toutes les données
learning_rate = 0.001

# Différentes étapes
for epo in range(num_epochs):
    loss_val = 0
    for data, target in index_pairs:
        x = Variable(get_input_layer(data)).float()
        y_true = Variable(torch.from_numpy(np.array([target])).long())

        # Matmul = produits matriciels de deux tensors
        z1 = torch.matmul(W1, x)
        z2 = torch.matmul(W2, z1)
    
    
        log_softmax = F.log_softmax(z2, dim=0)
        
        # nll_loss(pred/target)
        loss = F.nll_loss(log_softmax.view(1,-1), y_true)
        loss_val += loss.data
        
        # propagation
        loss.backward()
        W1.data -= learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data

        W1.grad.data.zero_()
        W2.grad.data.zero_()

In [9]:
W2

tensor([[ 2.7922e+00,  5.7247e-01, -6.3898e-01,  ..., -6.4101e-02,
          7.4126e-01, -4.9103e-01],
        [ 7.1234e-01, -1.2011e-02, -1.9086e+00,  ...,  2.1625e+00,
         -5.6097e-01, -1.4694e-01],
        [ 1.8898e+00,  5.2589e-01, -3.4958e-01,  ..., -7.8877e-01,
         -4.1727e-01, -9.9710e-01],
        ...,
        [-2.0081e+00,  2.7606e-01,  1.0070e-01,  ..., -5.1430e-01,
          3.4379e-02,  8.3176e-02],
        [ 3.5677e-01, -3.3767e-01, -2.8516e-01,  ...,  3.7814e-01,
         -1.1735e+00, -1.7944e-01],
        [-1.1754e+00,  1.0851e+00,  1.3936e-01,  ...,  1.8951e-03,
         -9.6587e-01,  7.0142e-01]], requires_grad=True)

In [10]:
# Distance/similarité cosinus
def cos_distance(u, v):
    return (np.dot(u, v)  / (math.sqrt(np.dot(u, u)) *  (math.sqrt(np.dot(v, v)))))

In [11]:
# Dictionnaire des poids
mot_poids = {index_mot[index]: poids.detach().numpy() for (index, poids) in enumerate(W2)}
mot_poids

{'il': array([ 2.7921708 ,  0.5724744 , -0.63897717,  0.3982683 , -0.42050698,
        -0.8936077 ,  1.1859536 ,  1.6140313 , -0.21710314,  0.24815668,
         0.37143105,  0.798329  ,  1.3293496 , -0.18132646, -1.0692025 ,
         0.686417  ,  0.61287886,  0.2950612 ,  1.4691409 ,  1.4364882 ,
        -0.6913636 ,  1.316635  ,  0.6220132 ,  1.5482095 , -0.06441711,
        -0.6066244 , -0.45783547, -0.52660704, -0.58989465, -0.23188917,
         2.4317963 , -1.6805742 ,  0.8739535 ,  2.222259  , -0.85222095,
         0.48514104, -0.09524805, -0.9243166 , -0.49171716, -0.36988068,
         0.14845772,  0.04062125, -0.18773669, -1.4577713 , -0.552582  ,
         0.06932555,  1.2609671 , -0.06410128,  0.74125767, -0.49103433],
       dtype=float32),
 'est': array([ 0.712342  , -0.01201111, -1.9085672 , -0.7600787 , -0.4927477 ,
        -1.9153293 , -0.3596756 , -2.0989509 , -0.41654035, -0.43527603,
         0.37066486, -0.55547583, -1.8901767 , -0.82331795,  0.42720863,
         0.481

### Étape 5 : Résultats du modèle

In [13]:
# Quelques résultats
cos_distance(mot_poids["roi"],mot_poids["reine"])

0.33133502105871776